In [1]:
import os
import pandas as pd
import numpy as np

# py-slippi
from slippi import Game

In [2]:
PATH='dataset/demo/inference'
SEQ_LEN=180

In [3]:
id_to_player = {0: '2saint',
 1: 'aklo',
 2: 'amsa',
 3: 'axe',
 4: 'cody',
 5: 'cpu',
 6: 'cynthia',
 7: 'hbox',
 8: 'kodorin',
 9: 'logan',
 10: 'mang0',
 11: 'moky',
 12: 'n0ne',
 13: 'plup',
 14: 'sfat',
 15: 'wizzrobe',
 16: 'yingling',
 17: 'zain'}

## Load the game

In [4]:
file = os.path.join(PATH, os.listdir(PATH)[0])
game = Game(file)
game

Game(
    end=End(
        lras_initiator=0,
        method=7:NO_CONTEST),
    frames=[...](2034),
    metadata=Metadata(
        console_name=None,
        date=2024-05-01 19:24:02+00:00,
        duration=2034,
        platform='dolphin':DOLPHIN,
        players=(
            Player(
                characters={9:PEACH: 2034},
                netplay=None),
            None,
            None,
            Player(
                characters={1:FOX: 2034},
                netplay=None))),
    start=Start(
        is_frozen_ps=False,
        is_pal=False,
        is_teams=False,
        players=(
            Player(
                character=12:PEACH,
                costume=0,
                stocks=4,
                tag=,
                team=None,
                type=0:HUMAN,
                ucf=UCF(
                    dash_back=1:UCF,
                    shield_drop=1:UCF)),
            None,
            None,
            Player(
                character=2:FOX,
                cos

## Get the inputs from the game

In [5]:
# get the port indices used by the players
def getPorts(game):
    ports = []
    for i in range(0, len(game.start.players)):
        if game.start.players[i] is not None:
            ports.append(i)

    return ports

# get the characters for each player, just for fun
def getCharacters(game):
    characters = []
    for player in game.start.players:
        if player is not None:
            characters.append(player.character)
    
    return characters

In [6]:
getCharacters(game)

[12:PEACH, 2:FOX]

In [7]:
START_FRAME = 64 # game starts on frame 64
COLS = ['joy_x', 'joy_y', 'cstick_x', 'cstick_y', 'z', 'r_dig', 'l_dig', 'a', 'b', 'x', 'y']
BUTTONS = ['Z', 'R', 'L', 'A', 'B', 'X', 'Y'] # names of the buttons in py-slippi

def getFrameInputs(player):
    # analog stick / c stick
    analog = [player.joystick.x, player.joystick.y, player.cstick.x, player.cstick.y]

    # buttons
    pressed_buttons = []
    # get the names of the buttons currently being pressed
    logical_pressed_names = map(lambda x: x.name, player.buttons.physical.pressed())

    for b in BUTTONS:

        if b in logical_pressed_names:
            pressed_buttons.append(1)
        else:
            pressed_buttons.append(0)

    return analog + pressed_buttons 

def getGameInputs(game):
    p1_inputs = pd.DataFrame(columns=COLS)
    p2_inputs = pd.DataFrame(columns=COLS)

    # get the controller port #s of the players
    p1_port, p2_port = getPorts(game)

    for i in range(START_FRAME, len(game.frames)):
        p1 = game.frames[i].ports[p1_port].leader.pre 
        p1_inputs.loc[len(p1_inputs.index)] = getFrameInputs(p1)

        p2 = game.frames[i].ports[p2_port].leader.pre
        p2_inputs.loc[len(p2_inputs.index)] = getFrameInputs(p2)

    return p1_inputs, p2_inputs

In [8]:
p1_inputs, _ = getGameInputs(game)
p1_inputs.head(10)


,joy_x,joy_y,cstick_x,cstick_y,z,r_dig,l_dig,a,b,x,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
p1_inputs.shape

(1970, 11)

## Compute the displacement vectors

In [10]:
def nextDisplacement(df, index):
    velocities = [] # stores both x and y displacement
    for col in df.columns:
        velocities.append(df.at[index+1, col] - df.at[index, col])

    return velocities

def gameDisplacements(game):
    
    ds = []
    prev_frame = 0
    for i in range(game.shape[0] - 1):
        # get next displacement
        d = nextDisplacement(game, i)

        if not d == [0, 0]: # only include frames that have some velocity
            # get elapsed
            elapsed = i - prev_frame
            prev_frame = i

            if elapsed > 60: # arbitrarily set all values > 60 to 0
                elapsed = 0

            ds.append((d[0], d[1], elapsed))
    
    return pd.DataFrame(ds, columns=['joy_x', 'joy_y', 'elapsed'])

In [11]:
displacements = gameDisplacements(p1_inputs[['joy_x', 'joy_y']])

In [12]:
displacements.head(10)

,joy_x,joy_y,elapsed
0,0.9875,0.0000,25
1,0.0125,0.0000,1
2,-0.0125,0.0000,39
3,-0.1000,-0.4500,1
4,-0.2375,-0.3000,1
5,-0.0750,-0.0625,1
6,-0.2375,-0.1125,1
7,-0.3375,-0.0750,1
8,0.0000,1.0000,2
9,0.0000,-0.4125,13


In [13]:
displacements.shape

(489, 3)

## Normalize elapsed values

In [14]:
max = 60

normed = displacements.copy()
normed['elapsed'] = normed['elapsed'].map(lambda x : x / max)

In [15]:
displacements.head(10)

,joy_x,joy_y,elapsed
0,0.9875,0.0000,25
1,0.0125,0.0000,1
2,-0.0125,0.0000,39
3,-0.1000,-0.4500,1
4,-0.2375,-0.3000,1
5,-0.0750,-0.0625,1
6,-0.2375,-0.1125,1
7,-0.3375,-0.0750,1
8,0.0000,1.0000,2
9,0.0000,-0.4125,13


## Process into sequences

In [16]:
def seqs_from_df(df, seq_len=SEQ_LEN):
    """
    Arrange a dataframe of inputs into sequences of inputs of length seq_len.
    """
    seqs = []

    for i in range(0, df.shape[0] - seq_len, int(seq_len*0.5)):
        seq_x = df.loc[i:i+seq_len-1]
        seqs.append(np.array(seq_x, dtype=np.float32))
        
    return seqs

In [17]:
sequences = seqs_from_df(normed)

print(f'{len(sequences)=}')
sequences[0][0:10]

len(sequences)=4


array([[ 0.9875    ,  0.        ,  0.41666666],
       [ 0.01249999,  0.        ,  0.01666667],
       [-0.01249999,  0.        ,  0.65      ],
       [-0.10000002, -0.45      ,  0.01666667],
       [-0.23750001, -0.3       ,  0.01666667],
       [-0.07499999, -0.0625    ,  0.01666667],
       [-0.23749998, -0.11250001,  0.01666667],
       [-0.3375    , -0.07499999,  0.01666667],
       [ 0.        ,  1.        ,  0.03333334],
       [ 0.        , -0.4125    ,  0.21666667]], dtype=float32)

## Load the pre-trained model

In [18]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, ELU, Dropout, Dense, Concatenate, LSTM

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      # allocated memory as needed, rather than 100% of it all the time
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2024-05-01 12:25:17.078014: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1 Physical GPUs, 1 Logical GPUs


2024-05-01 12:25:18.316240: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:838] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-01 12:25:18.333411: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:838] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-01 12:25:18.333462: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:838] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-01 12:25:18.334268: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:838] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-01 12:25:18.334333: I tensorflow/compiler/xla/stream_executo

In [19]:
def createClassifier(width=3, seq_len=180):
    input_layer = Input(shape=(seq_len, width))
    conv1 = Conv1D(filters=32, kernel_size=7, strides=2, activation=ELU())(input_layer)
    conv2 = Conv1D(filters=32, kernel_size=3, strides=1, activation=ELU())(input_layer)

    catted = Concatenate(axis=1)([conv1, conv2])
    elu1 = ELU(32)(catted)
    conv3 = Conv1D(filters=32, kernel_size=2, strides=1, activation=ELU())(elu1)
    conv4 = Conv1D(filters=32, kernel_size=2, strides=1, activation=ELU())(conv3)
    drop1 = Dropout(0.2)(conv4)

    gru1 = LSTM(32, return_sequences=True)(drop1)
    gru2 = LSTM(32)(gru1)
    drop2 = Dropout(0.2)(gru2)

    output = Dense(18, activation='softmax')(drop2)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
model = createClassifier(3, 180)

model.load_weights('models/demo')
model.summary()

2024-05-01 12:25:18.535802: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.537368: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.592117: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.594839: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.597318: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.599260: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.601032: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.602823: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.603400: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 180, 3)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 87, 32)               704       ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 178, 32)              320       ['input_1[0][0]']             
                                                                                                  
 concatenate (Concatenate)   (None, 265, 32)              0         ['conv1d[0][0]',              
                                                                     'conv1d_1[0][0]']        

2024-05-01 12:25:18.933332: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.934053: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.934841: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.935347: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.938672: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.939249: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.939745: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.942144: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.942640: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18

In [21]:
predicts = model.predict(np.array(sequences, dtype=np.float32))

hard_predicts = [np.argmax(predicts[i]) for i in range(predicts.shape[0])]

[id_to_player[hp] for hp in hard_predicts]

2024-05-01 12:25:18.989344: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.990292: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.995658: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:18.996825: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:19.005211: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:19.006358: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:19.007332: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:19.008740: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:19.013667: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2024-05-01 12:25:19

1/1 [==============================] - 1s 1s/step


['axe', 'cynthia', 'amsa', 'cynthia']